In [18]:
import numpy as np
import pandas as pd

In [19]:
dataset = pd.read_csv("Data.csv")

In [20]:
display(dataset)

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [22]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 3].values

In [23]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [24]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## SimpleImputer
<p>Ponieważ użycie klasy Imputer wyświetla informację o deprecjacji, użyta jest sugerowana klasa SimpleImputer. W tym przypadku zamiana wymagała tylko usunięcia parametru *axis*.</p>
<p>Przy pomocy Imputera uzupełnia się brakujące wartości w tabeli.</p>
<p>Parametry:
<ul>
    <li><b>missing_values</b> - w tym wypadku wybiera oznaczone jako "NaN" (not a number)</li>
    <li><b>strategy</b> - <i>mean</i>, czyli za brakujące wartości wstawiana jest wartość średnia z pozostałych wartości w kolumnie</li>
</ul>
Metody:
<ul>
    <li><b>fit</b> - przetwarza dane wejściowe zgodnie z uzytymi parametrami</li>
    <li><b>transform</b> - zapisuje tabelę z przetworzonymi danymi</li>
    <li><b>fit_transform</b> - łączy obie powyższe metody</li>
</ul></p>

In [25]:
from sklearn.impute import SimpleImputer

In [26]:
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
X[:, 1:3] = imputer.fit_transform(X[:, 1:3])

In [27]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## ColumnTransformer

#### Oryginalnie użyte klasy
<ul>
    <li><b>LabelEncoder</b> - przetwarza powtarzajace się dane (etykiety) na liczby 0 - liczba_etykiet-1 zarówno dla danych liczbowych, jak i tekstowych.<br />
    metoda fit przypisuje unikalnym etykietom numery od 0<br />
    metoda transform zamienia etykiety na ich numeryczne odpowiedniki w tabeli</li>
    <li><b>OneHotEncoder</b> - przetwarza etykiety na tabelę binarną one-hot, czyli dla każdej etykiety tworzona jest osobna kolumna z wartościami 0 albo 1 (1 w wierszach, gdzie była dana etykieta)</li>
</ul>
    
#### Zmiana
<p>Wykorzystanie oryginalnego kodu wyświetlało dwie informacje:</p>
<ul>
    <li><i>FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values. If you want the future behaviour and silence this warning, you can specify "categories='auto'". In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.</i></li>
    <li><i>The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.</i></li>
</ul>
<p>Pierwsza z nich oznacza, że wcześniejsze użycie LabelEncodera nie jest potrzebne.
Druga, że jednen z parametrów, pozwalający na wybranie przetwarzanej kolumny, przestaje być używany, oraz sugeruje użycie klasy <b>ColumnTransformer</b> z pakietu <b>sklearn.compose</b>.</p>
<p><b>ColumnTransformer</b> - pozwala na użycie różnych transformacji, jak OneHotEncoder w jednym kroku na wybranych kolumnach. </p>

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [29]:
ct = ColumnTransformer([("OneHotEncode_X", OneHotEncoder(categories="auto"),[0])], remainder="passthrough")
X = ct.fit_transform(X)  

In [30]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Kodowanie etykiet y

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [33]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


### Podział danych na zestawy treningowy i testowy
<p>Zestaw treningowy służy do uczenia modelu<br />
Zestaw testowy służy do ostatecznego sprawdzenia modelu i powinien składać się z danych nie używanych w treningu czy walidacji</p>

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [36]:
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (8, 5)
X_test:  (2, 5)
y_train:  (8,)
y_test:  (2,)


### Standaryzacja danych
<p>Wykonywana, żeby rozkład danych przypominał rozkład normalny ze średnią w 0.</p>

In [37]:
from sklearn.preprocessing import StandardScaler

In [38]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [39]:
print(X_train)

[[-1.          2.64575131 -0.77459667  0.26306757  0.12381479]
 [ 1.         -0.37796447 -0.77459667 -0.25350148  0.46175632]
 [-1.         -0.37796447  1.29099445 -1.97539832 -1.53093341]
 [-1.         -0.37796447  1.29099445  0.05261351 -1.11141978]
 [ 1.         -0.37796447 -0.77459667  1.64058505  1.7202972 ]
 [-1.         -0.37796447  1.29099445 -0.0813118  -0.16751412]
 [ 1.         -0.37796447 -0.77459667  0.95182631  0.98614835]
 [ 1.         -0.37796447 -0.77459667 -0.59788085 -0.48214934]]


In [40]:
print(X_test)

[[-1.          2.64575131 -0.77459667 -1.45882927 -0.90166297]
 [-1.          2.64575131 -0.77459667  1.98496442  2.13981082]]
